In [1]:
import urllib

ACCESS_KEY = "xxxxxxxxxxx"
SECRET_KEY = "xxxxxxxxxxx"
ENCODED_SECRET_KEY = urllib.quote(SECRET_KEY, "")
AWS_BUCKET_NAME = "rohit-sparkfish"
MOUNT_NAME = "sparkfish"
dbutils.fs.mount("s3n://%s:%s@%s" % (ACCESS_KEY, ENCODED_SECRET_KEY, AWS_BUCKET_NAME), "/mnt/%s" % MOUNT_NAME)
display(dbutils.fs.ls("/mnt/sparkfish"))

In [2]:
%python
from pyspark.sql import functions as F
from pyspark.sql.functions import datediff, to_date, lit, unix_timestamp,split
from pyspark.sql.types import *

# Build DataFrame dataset to work with. 
formatPackage = "csv" if sc.version > '1.6' else "com.databricks.spark.csv"
df = sqlContext.read.format(formatPackage).options(header='true', delimiter = ',').load("dbfs:/mnt/sparkfish/titanic.csv")
data_df=df.withColumn("Age", df["Age"].cast(IntegerType()))
data_df.printSchema()
data_df.write.saveAsTable('sparkfishTable', format='parquet', mode='overwrite',path='dbfs:/mnt/sparkfish/sparkfishTable/')

In [3]:
from pyspark import SparkContext, HiveContext
hiveContext = HiveContext(sc)
display(hiveContext.sql("SELECT percentile(Age, 0.75) FROM sparkfishTable"))


In [4]:
from pyspark import SparkContext, HiveContext
hiveContext = HiveContext(sc)
display(hiveContext.sql("SELECT avg(Age) FROM sparkfishTable"))